In [1]:
import json
import regex
from unidecode import unidecode
import yaml
import logging
from logging import config
import numpy as np 
import pandas as pd
from IPython.display import Markdown
from datetime import datetime as dt
from time import sleep

import spacy
from spacy import displacy
spacy.require_gpu()
from torch.utils import dlpack

import seaborn as sns

In [2]:
import importlib.util as imp
import sys

spec_conn = imp.spec_from_file_location(
    'twitter_connection',
    '../twitter-connection/__init__.py')
sc = imp.module_from_spec(spec_conn)
sys.modules[spec_conn.name] = sc
spec_conn.loader.exec_module(sc)

spec_data = imp.spec_from_file_location(
    'twitter_data',
    '../twitter-connection/twitter_data/__init__.py')
twit = imp.module_from_spec(spec_data)
sys.modules[spec_data.name] = twit
spec_data.loader.exec_module(twit)

from twitter_connection import processing
from twitter_connection.util import utils
from twitter_data import twitter_data, tweets, users, places

In [3]:
try:
    with open(utils.get_project_root()/'log_config.yml', 'r') as f:
        lc = yaml.safe_load(f)
        config.dictConfig(lc)

        logger = logging.getLogger('processing')
except Exception as e:
    print(e.args)

In [4]:
gen_conf = utils.get_config()
conf = utils.get_config('p')

In [5]:
nlp_es = spacy.load(conf['spacy']['es'], disable=conf['spacy']['pipeline']['disable'])

In [6]:
es_save_file = 'tweets-processed-combined.csv'
es_save_paths = [utils.get_save_path('p', lang='es')/'07112021-at-2210']

verb_conjug_path = utils.get_project_root()/gen_conf['file_paths']['verb_conjug']

In [7]:
verbs = pd.read_excel(verb_conjug_path)
verbs.head(2)

,verb_type,verb,indicativo,imperativo,subjuntivo,gerundio,gerundio_compuesto,infinitivo,infinitivo_compuesto,participio_pasado
0,Stative,ver,veía visto verías vi vimos verían ves v...,vean ve vea veamos ved,veáis visto vieras vieren viesen veas vi...,viendo,visto,ver,visto,visto
1,Stative,jurar,jurarán juramos jurarías jurabas juraría ...,jurad jura juren jure juremos,jurare jurareis jurase jurara juraren jur...,jurando,jurado,jurar,jurado,jurado


import importlib
importlib.reload(utils)

In [8]:
data_list = []

for path in es_save_paths:
    print(path/es_save_file)
    data_list.append(utils.get_csv(path/es_save_file))

/home/rimov/Documents/Code/NLP/lin-que-dropping/processing/../processing/saved/es/07112021-at-2210/tweets-processed-combined.csv


In [9]:
for d in data_list:
    print(d.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298492 entries, 0 to 298491
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   tweet_id                  298492 non-null  float64
 1   verbs                     298492 non-null  string 
 2   text_orig                 298492 non-null  string 
 3   text_norm                 298492 non-null  string 
 4   dependencies              298492 non-null  string 
 5   lemma_pos_stopword        298492 non-null  string 
 6   retweet_reply_like_quote  298492 non-null  string 
 7   created_at                298492 non-null  object 
 8   user_id                   298492 non-null  string 
 9   tweet_place_id            298491 non-null  string 
 10  mentions                  185057 non-null  string 
 11  referenced_tweets         180174 non-null  string 
dtypes: float64(1), object(1), string(10)
memory usage: 27.3+ MB
None


In [10]:
data = pd.concat(data_list, ignore_index=True)

In [39]:
not_na = ~data['text_norm'].isna()
data = data[not_na].copy()

In [42]:
tweet_metadata = data.loc[:, conf['metadata_cols']]
tweet_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 298492 entries, 0 to 298491
Data columns (total 7 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   tweet_id                  298492 non-null  float64
 1   created_at                298492 non-null  object 
 2   retweet_reply_like_quote  298492 non-null  string 
 3   user_id                   298492 non-null  string 
 4   tweet_place_id            298491 non-null  string 
 5   mentions                  185057 non-null  string 
 6   referenced_tweets         180174 non-null  string 
dtypes: float64(1), object(1), string(5)
memory usage: 18.2+ MB


In [43]:
meta_path = es_save_paths[0]/'metadata'
utils.make_dir(meta_path)

PosixPath('/home/rimov/Documents/Code/NLP/lin-que-dropping/processing/../processing/saved/es/07112021-at-2210/metadata')

In [44]:
utils.save_excel(meta_path, tweet_metadata, 'tweet_metadata')

In [36]:
data = data.loc[:, conf['analysis_cols']]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 298492 entries, 0 to 298491
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tweet_id            298492 non-null  float64
 1   verbs               298492 non-null  string 
 2   text_orig           298492 non-null  string 
 3   text_norm           298492 non-null  string 
 4   dependencies        298492 non-null  string 
 5   lemma_pos_stopword  298492 non-null  string 
dtypes: float64(1), string(5)
memory usage: 15.9 MB


In [37]:
has_na = data.isna().sum()
has_na

tweet_id              0
verbs                 0
text_orig             0
text_norm             0
dependencies          0
lemma_pos_stopword    0
dtype: int64

In [14]:
data['text_norm'] = data['text_norm'].apply(unidecode)

In [15]:
samples = (np.random.rand(10)*data.shape[0]).astype(int)
sample_text = data.loc[samples, 'text_norm'].to_numpy()
sample_text

array(['No es funcion del gobierno brindar mas seguridad???  Es un reverendo hijo de puta... si el gobierno no garantiza mi integridad fisica como ciudadano, entonces consigo un FAL y me cargo unos cuantos indios K',
       'Imagina que yo soy tu novio cuando me bese',
       'La situacion de adulterio es horrible, te sentis tan mal, impotente, pero quiza le queres tanto a esa persona que dejas pasar de largo la situacion mas siendo mami de familia una tiene que bancarse x los chicos y asi.. . Ojala pronto salga de esa relacion, no le conviene',
       'Correcto, te deseo una linda noche llena de bonitos suenos, saludos a tu mama.',
       'Supongo que no habra vaso de recuerdo',
       'Yo solo le pido a Dios diariamente que me cuide a mi familia, nada mas ',
       'Estas imagenes son antiguas, para un proyecto que presentamos al Ministerio, pero me las recuerda Google ahora y me han gustado, casi ni me acordaba.',
       'El sexo no es pecado ni vicio. Si te satisface y el cuerpo te

In [16]:
def test_text_has_match(text, search:list):
    present = set()
    
    for term in search:
        mat = regex.search(term, text.lower())
        
        if mat is not None:
            present.add(mat.group())
    
    return len(present)==len(search)

In [17]:
test_text_has_match(sample_text[0], [r'\bquerido\b', r'\basi\b'])

False

In [18]:
data['has_ccomp'] = data['dependencies'].str.contains('ccomp')
print(data['has_ccomp'].sum())
display(data.head(2))

133144


,tweet_id,verbs,text_orig,text_norm,dependencies,lemma_pos_stopword,has_ccomp
0,1.451193e+18,"sentir, pedir",".@CitroenEspana Cactus con 5,5 años. Me empiez...",".Cactus con 5,5 anos. Me empieza a salir oxido...",".Cactus(ROOT) con(case) 5,5(nummod) anos(nmod)...",.Cactus(.Cactus|PROPN|False) con(con|ADP|True)...,False
1,1.451193e+18,sentir,"Me toy bebiendo un té, y siento como que toy s...","Me toy bebiendo un te, y siento como que toy s...",Me(iobj) toy(ROOT) bebiendo(xcomp) un(det) te(...,Me(yo|PRON|True) toy(tar|VERB|False) bebiendo(...,True


In [19]:
data['has_que'] = data['text_norm'].str.contains(r'\bque\b|\bq\b', flags=regex.IGNORECASE)
data['has_que'].sum()

199199

In [20]:
col_order = ['verbs', 'text_orig', 'dependencies', 'has_ccomp', 'has_que', 'text_norm', 'lemma_pos_stopword', 'tweet_id']
data = data.loc[:, col_order]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 298492 entries, 0 to 298491
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   verbs               298492 non-null  string 
 1   text_orig           298492 non-null  string 
 2   dependencies        298492 non-null  string 
 3   has_ccomp           298492 non-null  boolean
 4   has_que             298492 non-null  bool   
 5   text_norm           298492 non-null  object 
 6   lemma_pos_stopword  298492 non-null  string 
 7   tweet_id            298492 non-null  float64
dtypes: bool(1), boolean(1), float64(1), object(1), string(4)
memory usage: 26.8+ MB


In [30]:
import importlib
importlib.reload(processing)

<module 'twitter_connection.processing' from '/home/rimov/Documents/Code/NLP/lin-que-dropping/processing/../twitter-connection/processing.py'>

In [31]:
time = dt.now().strftime('%d%m%Y-at-%H%M')

In [32]:
save_path = es_save_paths[0]/'processed-latest'
utils.make_dir(save_path)
save_path = save_path/time
utils.make_dir(save_path)

PosixPath('/home/rimov/Documents/Code/NLP/lin-que-dropping/processing/../processing/saved/es/07112021-at-2210/processed-latest/14112021-at-1609')

In [33]:
processing.save_by_verb(data, verb_conjug_path, save_path, save_file='excel')

In [24]:
cleaned = data.dropna(subset=['text_norm'])
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 754170 entries, 0 to 755062
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype              
---  ------                    --------------   -----              
 0   tweet_id                  754170 non-null  object             
 1   verbs                     754170 non-null  object             
 2   text_orig                 754170 non-null  object             
 3   text_norm                 754170 non-null  object             
 4   dependencies              753954 non-null  object             
 5   lemma_pos_stopword        753882 non-null  object             
 6   retweet_reply_like_quote  753923 non-null  object             
 7   created_at                753863 non-null  datetime64[ns, UTC]
 8   author_id                 360104 non-null  float64            
 9   place_id                  360104 non-null  object             
 10  mentions                  484384 non-null  object             
 11  

In [21]:
vs = set(verbs['verb'].to_numpy())
vs

{'acordar',
 'adivinar',
 'admitir',
 'afirmar',
 'apostar',
 'asegurar',
 'comprobar',
 'confesar',
 'confirmar',
 'conseguir',
 'considerar',
 'contar',
 'creer',
 'decir',
 'demostrar',
 'desear',
 'dudar',
 'entender',
 'esperar',
 'gritar',
 'imaginar',
 'jurar',
 'lamentar',
 'lograr',
 'mandar',
 'mencionar',
 'mostrar',
 'negar',
 'ojala',
 'ordenar',
 'parecer',
 'pedir',
 'pensar',
 'predecir',
 'prever',
 'prometer',
 'querer',
 'reclamar',
 'recomendar',
 'recordar',
 'responder',
 'rogar',
 'saber',
 'sentir',
 'solicitar',
 'suplicar',
 'suponer',
 'suspirar',
 'temer',
 'ver'}

In [39]:
cleaned['verbs'] = cleaned['verbs'].apply(lambda x: regex.sub(r'[^\w]+', '', x)).copy()
cleaned.head(3)

/tmp/ipykernel_15056/939276038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned['verbs'] = cleaned['verbs'].apply(lambda x: regex.sub(r'[^\w]+', '', x)).copy()


,tweet_id,verbs,text_orig,text_norm,dependencies,lemma_pos_stopword,retweet_reply_like_quote,created_at,author_id,place_id,mentions,referenced_tweets,user_id,tweet_place_id
0,1.41946e+18,saber,@Velasqu08012862 @NativiManuel @nayibbukele En...,"En la declaración de patrimonio, se detallaban...",En(case) la(det) declaración(obl) de(case) pat...,En(en|ADP|True) la(el|DET|True) declaración(de...,"(0, 0, 1, 0)",2021-07-26 00:46:01+00:00,1.334955e+18,01087f69071d020f,"['1409932585728417802', '893205667405844481', ...",['1419445124120072192'],NaN,NaN
1,1.41946e+18,demostrar,@atlanticsurff La misma m... de siempre lo son...,La misma m... de siempre lo son y lo demuestran.,La(det) misma(det) m(nsubj) ...() de(advmod) s...,La(el|DET|True) misma(mismo|DET|True) m(m|NOUN...,"(0, 0, 0, 0)",2021-07-26 00:43:15+00:00,1.581253e+09,018f1cde6bad9747,['141986676'],['1419297031445749760'],NaN,NaN
2,1.41946e+18,sentir,"¿Alguien más se siente, triste, insuficiente, ...","¿Alguien más se siente, triste, insuficiente, ...",¿() Alguien(nsubj) más(advmod) se(obj) siente(...,Alguien(alguien|PRON|False) más(más|ADV|True) ...,"(0, 0, 0, 0)",2021-07-26 00:36:01+00:00,2.901198e+09,13d479b108707983,NaN,NaN,NaN,NaN


In [40]:
bad = ~cleaned['verbs'].isin(vs)
bad.sum()

272

In [41]:
final = cleaned.drop(cleaned[bad].index)
final.head()

,tweet_id,verbs,text_orig,text_norm,dependencies,lemma_pos_stopword,retweet_reply_like_quote,created_at,author_id,place_id,mentions,referenced_tweets,user_id,tweet_place_id
0,1.41946e+18,saber,@Velasqu08012862 @NativiManuel @nayibbukele En...,"En la declaración de patrimonio, se detallaban...",En(case) la(det) declaración(obl) de(case) pat...,En(en|ADP|True) la(el|DET|True) declaración(de...,"(0, 0, 1, 0)",2021-07-26 00:46:01+00:00,1.334955e+18,01087f69071d020f,"['1409932585728417802', '893205667405844481', ...",['1419445124120072192'],NaN,NaN
1,1.41946e+18,demostrar,@atlanticsurff La misma m... de siempre lo son...,La misma m... de siempre lo son y lo demuestran.,La(det) misma(det) m(nsubj) ...() de(advmod) s...,La(el|DET|True) misma(mismo|DET|True) m(m|NOUN...,"(0, 0, 0, 0)",2021-07-26 00:43:15+00:00,1.581253e+09,018f1cde6bad9747,['141986676'],['1419297031445749760'],NaN,NaN
2,1.41946e+18,sentir,"¿Alguien más se siente, triste, insuficiente, ...","¿Alguien más se siente, triste, insuficiente, ...",¿() Alguien(nsubj) más(advmod) se(obj) siente(...,Alguien(alguien|PRON|False) más(más|ADV|True) ...,"(0, 0, 0, 0)",2021-07-26 00:36:01+00:00,2.901198e+09,13d479b108707983,NaN,NaN,NaN,NaN
3,1.41946e+18,decir,Me dicen que cuando el m-19 se robó la espada ...,Me dicen que cuando el m-19 se robó la espada ...,Me(iobj) dicen(ROOT) que(mark) cuando(mark) el...,Me(yo|PRON|True) dicen(decir|VERB|True) que(qu...,"(0, 0, 3, 0)",2021-07-26 00:33:48+00:00,9.815425e+08,00d7fce06c2ec290,NaN,NaN,NaN,NaN
4,1.41946e+18,ver,"bueno de perdida si no me siento bien, por lo ...","bueno de perdida si no me siento bien, por lo ...",bueno(ROOT) de(case) perdida(nmod) si(mark) no...,bueno(bueno|ADJ|True) de(de|ADP|True) perdida(...,"(0, 2, 4, 0)",2021-07-26 00:33:20+00:00,1.000247e+18,22886de3634e157a,NaN,NaN,NaN,NaN


In [42]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 753898 entries, 0 to 755062
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype              
---  ------                    --------------   -----              
 0   tweet_id                  753898 non-null  object             
 1   verbs                     753898 non-null  object             
 2   text_orig                 753898 non-null  object             
 3   text_norm                 753898 non-null  object             
 4   dependencies              753878 non-null  object             
 5   lemma_pos_stopword        753878 non-null  object             
 6   retweet_reply_like_quote  753863 non-null  object             
 7   created_at                753863 non-null  datetime64[ns, UTC]
 8   author_id                 360104 non-null  float64            
 9   place_id                  360104 non-null  object             
 10  mentions                  484384 non-null  object             
 11  

In [44]:
vdistrib = final['verbs'].value_counts()

In [46]:
d = vdistrib.to_dict()

In [47]:
d

{'decir': 88473,
 'ver': 85622,
 'querer': 63515,
 'parecer': 53834,
 'saber': 44011,
 'entender': 42622,
 'sentir': 40128,
 'creer': 37700,
 'pedir': 30433,
 'mandar': 24777,
 'pensar': 22382,
 'esperar': 21893,
 'contar': 20887,
 'recordar': 20479,
 'imaginar': 13237,
 'desear': 11418,
 'lograr': 10332,
 'acordar': 10301,
 'responder': 9597,
 'demostrar': 8365,
 'suponer': 8224,
 'conseguir': 7647,
 'mostrar': 7129,
 'jurar': 6172,
 'considerar': 5551,
 'dudar': 5366,
 'recomendar': 5216,
 'negar': 4777,
 'confirmar': 4627,
 'gritar': 4394,
 'rogar': 3677,
 'prometer': 3563,
 'mencionar': 3560,
 'asegurar': 3370,
 'lamentar': 2724,
 'reclamar': 2697,
 'temer': 2428,
 'apostar': 2413,
 'solicitar': 2306,
 'afirmar': 1663,
 'admitir': 1526,
 'ordenar': 1362,
 'confesar': 962,
 'comprobar': 924,
 'adivinar': 791,
 'predecir': 270,
 'prever': 248,
 'suspirar': 171,
 'suplicar': 108,
 'ojala': 26}

In [9]:
try:
    data = pd.concat([pd.read_csv(path/es_save_file, 
                           sep='~', 
                           parse_dates=gen_conf['dtypes']['dates']) for path in es_save_paths], ignore_index=True)
#     data = data.drop_duplicates('tweet_id').reset_index(drop=True)
except ValueError as ve:
    print(ve.args)

/home/rimov/anaconda3/envs/nlp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (1,2,3,4,5,6,9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5258491 entries, 0 to 5258490
Data columns (total 14 columns):
 #   Column                    Dtype              
---  ------                    -----              
 0   tweet_id                  object             
 1   verbs                     object             
 2   text_orig                 object             
 3   text_norm                 object             
 4   dependencies              object             
 5   lemma_pos_stopword        object             
 6   retweet_reply_like_quote  object             
 7   created_at                datetime64[ns, UTC]
 8   author_id                 float64            
 9   place_id                  object             
 10  mentions                  object             
 11  referenced_tweets         object             
 12  user_id                   float64            
 13  tweet_place_id            object             
dtypes: datetime64[ns, UTC](1), float64(2), object(11)
memory usage: 56

In [17]:
has_que = data['text_norm'].str.contains(r'\bque\b|\bq\b')
print(f'Have "que": {has_que.sum()}, do not: {data.shape[0]-has_que.sum()}')

Have "que": 192265, do not: 106299


In [23]:
mixed_type = data[data['text_norm'].dtype==str]

KeyError: False

In [ ]:
data_que = data[has_que]
data_nque = data[~has_que]

In [31]:
len(sample_none)

27

In [32]:
proc_sample_que = sample_que['text_normd'].apply(nlp_es)
proc_sample_none = sample_none['text_normd'].apply(nlp_es)

In [104]:
data['word_pos'].iloc[0]

"{'la': 'DET', 'miseria': 'NOUN', 'de': 'ADP', 'salarios': 'NOUN', 'no': 'ADV', 'espera': 'VERB', 'analisis': 'NOUN', 'tecnicos': 'ADJ', 'son': 'AUX', 'unos': 'DET', 'buitres': 'NOUN', 'estos': 'DET', 'empresarios': 'NOUN', 'explotadores': 'ADJ'}"

#### Sample with 'que'

In [53]:
options = {'compact': True, 'word_spacing': 45, 'distance': 100, 'add_lemma': True, 'fine_grained': True}

In [56]:
trees_que = displacy.render(proc_sample_que, options=options, jupyter=False)

In [58]:
with open('/home/rimov/Documents/Code/NLP/lin-que-dropping/processing/images/trees_sample_que.html', 'w') as f:
    f.write(trees_que)

#### Sample without 'que'

In [59]:
trees_nque = displacy.render(proc_sample_none, options=options, jupyter=False)

In [60]:
with open('/home/rimov/Documents/Code/NLP/lin-que-dropping/processing/images/trees_sample_nque.html', 'w') as f:
    f.write(trees_nque)

#### Separate Data

In [66]:
dep = data['text_normd'].apply(nlp_es)

In [67]:
def get_dep(procd):
    return {p.text.lower(): p.dep_ for p in procd}

In [84]:
def has_ccomp(procd):
    deps = set(p.dep_ for p in procd)
    return 'ccomp' in deps

In [85]:
data_deps = dep.apply(get_dep).rename('dependencies')
data_deps.head()

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               {'la': 'det', 'miseria': 'nsubj', 'de': 'case', 'salarios': 'nmod', 'no': 'advmod', 'espera': 'ROOT', 'analisis': 'obj', 'tecnicos': 'amod', ',': 'punct', 'son': 'cop', 'unos': 'det', 'buitres': 'advcl', 'estos': 'det', 'empresarios': 'nsubj', 'explotadores': 'amod', '.': 'punct'}
1    {'estaba': 'ROOT', 'rodrigo': 'nsubj', 'del': 'case', 'leeds': 'nmod', 'pero': 'advmod', 'es': 'cop', 'verdad': 'conj', 'que': 'nsubj', 'no': 'advmod', 'ha': 'aux', 'llegado': 'csubj', 'a': 'case', 'se

In [92]:
type(data_deps.iloc[0])

dict

In [86]:
data_has_ccomp = dep.apply(has_ccomp).rename('has_ccomp')
data_has_ccomp.head()

0    False
1     True
2    False
3     True
4     True
Name: has_ccomp, dtype: bool

In [98]:
# Rearrange columns
visual_order = ['query_topic', 
                'verbs',
                'text_orig', 
                'lemmad',
                'word_pos', 
                'dependencies',
                'has_ccomp',
                'retweet_reply_like_quote', 
                'created_at', 
                'tweet_location', 
                'user_location', 
                'username',
                'user_tweet_count',
                'followers_following_listed',
                'tweet_id']

In [99]:
conct = pd.concat([data, data_deps, data_has_ccomp], axis=1)
conct = conct.loc[:, visual_order]
conct.head()

,query_topic,verbs,text_orig,lemmad,word_pos,dependencies,has_ccomp,retweet_reply_like_quote,created_at,tweet_location,user_location,username,user_tweet_count,followers_following_listed,tweet_id
0,esperar,['esperar'],"@Teleprensa33 @camarasal La miseria de salarios no espera análisis técnicos, son unos buitres estos empresarios explotadores.",el miseria de salario no esperar analisi tecnico ser uno buitre este empresario explotador,"{'la': 'DET', 'miseria': 'NOUN', 'de': 'ADP', 'salarios': 'NOUN', 'no': 'ADV', 'espera': 'VERB', 'analisis': 'NOUN', 'tecnicos': 'ADJ', 'son': 'AUX', 'unos': 'DET', 'buitres': 'NOUN', 'estos': 'DET', 'empresarios': 'NOUN', 'explotadores': 'ADJ'}","{'la': 'det', 'miseria': 'nsubj', 'de': 'case', 'salarios': 'nmod', 'no': 'advmod', 'espera': 'ROOT', 'analisis': 'obj', 'tecnicos': 'amod', ',': 'punct', 'son': 'cop', 'unos': 'det', 'buitres': 'advcl', 'estos': 'det', 'empresarios': 'nsubj', 'explotadores': 'amod', '.': 'punct'}",False,"(0, 0, 0, 0)",2021-07-06 22:12:54,"Ayutuxtepeque, El Salvador","San Salvador, El Salvador",quintanilla503s,4196,"(108, 685, 0)",1412535051275940096
1,esperar,['esperar'],"@lavigi @Ramarmo Estaba Rodrigo del Leeds pero es verdad que no ha llegado a ser lo que se esperaba. \nLo que le falta es un delantero alto con movilidad para salir de la posición y crear espacios a su espalda. Pero no hay nada ningún proyecto de ese tipo, ahora mismo (Rafa Mira?).",estar Rodrigo del Leeds pero ser verdad que no haber llegar a ser él que él esperar \n él que él faltar ser uno delantero alto con movilidad para salir de el posicion y crear espacio a su espalda pero no haber nada ningun proyecto de ese tipo ahora mismo Rafa Mira,"{'estaba': 'VERB', 'rodrigo': 'PROPN', 'del': 'ADP', 'leeds': 'PROPN', 'pero': 'CCONJ', 'es': 'AUX', 'verdad': 'NOUN', 'que': 'PRON', 'no': 'ADV', 'ha': 'AUX', 'llegado': 'VERB', 'a': 'ADP', 'ser': 'AUX', 'lo': 'PRON', 'se': 'PRON', 'esperaba': 'VERB', '\n': 'SPACE', 'le': 'PRON', 'falta': 'VERB', 'un': 'DET', 'delantero': 'NOUN', 'alto': 'ADJ', 'con': 'ADP', 'movilidad': 'NOUN', 'para': 'ADP', 'salir': 'VERB', 'de': 'ADP', 'la': 'DET', 'posicion': 'NOUN', 'y': 'CCONJ', 'crear': 'VERB', 'espacios': 'NOUN', 'su': 'DET', 'espalda': 'NOUN', 'hay': 'AUX', 'nada': 'PRON', 'ningun': 'DET', 'proyecto': 'NOUN', 'ese': 'DET', 'tipo': 'NOUN', 'ahora': 'ADV', 'mismo': 'ADV', 'rafa': 'PROPN', 'mira': 'PROPN'}","{'estaba': 'ROOT', 'rodrigo': 'nsubj', 'del': 'case', 'leeds': 'nmod', 'pero': 'advmod', 'es': 'cop', 'verdad': 'conj', 'que': 'nsubj', 'no': 'advmod', 'ha': 'aux', 'llegado': 'csubj', 'a': 'case', 'ser': 'xcomp', 'lo': 'det', 'se': 'expl:pass', 'esperaba': 'ccomp', '.': 'punct', ' ': 'punct', 'le': 'obj', 'falta': 'csubj', 'un': 'det', 'delantero': 'ROOT', 'alto': 'amod', 'con': 'case', 'movilidad': 'nmod', 'para': 'mark', 'salir': 'acl', 'de': 'case', 'la': 'det', 'posicion': 'obl', 'y': 'cc', 'crear': 'conj', 'espacios': 'obj', 'su': 'det', 'espalda': 'obl', 'hay': 'ROOT', 'nada': 'obj', 'ningun': 'det', 'proyecto': 'obj', 'ese': 'det', 'tipo': 'nmod', ',': 'punct', 'ahora': 'advmod', 'mismo': 'advmod', '(': 'punct', 'rafa': 'dep', 'mira': 'flat', '?': 'punct', ')': 'punct'}",True,"(0, 1, 2, 0)",2021-07-06 22:11:57,"Paris, France","Paris, France",AlmendraPeleona,14057,"(1097, 354, 8)",1412534812712260096
2,esperar,['esperar'],@SuryZury01 Bueno espero serlo,bueno esperar ser él,"{'bueno': 'INTJ', 'espero': 'VERB', 'serlo': 'AUX'}","{'bueno': 'dep', 'espero': 'ROOT', 'serlo': 'xcomp'}",False,"(0, 0, 2, 0)",2021-07-06 22:10:37,Honduras,"Tegucigalpa, Honduras",mrhombre_,15113,"(4657, 1217, 2)",1412534477876781056
3,esperar,['esperar'],"Yo, como la mayoría de nosotros, no esperaba que la selección llegara tan lejos en esta Eurocopa. Y vaya, vaya si lo han hecho. Ya me había hecho ilusiones, me veía en la final. Gracias por hacernos disfrutar. Orgulloso de ser español. Viva España. 💪🇪🇦",yo como el mayoria de yo no esperar que el seleccion llegar tanto lejos en este Euroco

In [101]:
conct.shape[0]

43769

In [100]:
conct.to_excel('/home/rimov/Documents/Code/NLP/lin-que-dropping/processing/saved/es/twitter-es-esperar-postagged-20-7-2021.xlsx', index=False)

### Spacy Details

In [62]:
terms = ['csubj', 'dep', 'cc', 'ccomp', 'xcomp']

In [65]:
for t in terms:
    print(spacy.explain(t))

clausal subject
unclassified dependent
coordinating conjunction
clausal complement
open clausal complement


### Normalize Text (optional)
If failed to save normalized version

In [46]:
normalize = False

In [36]:
if normalize:
    norm = tweets.Tweets(data).normalized
    display(norm[:2])

,tweet_id,text_normd
0,1412535051275940096,"La miseria de salarios no espera analisis tecnicos, son unos buitres estos empresarios explotadores."
1,1412534812712260096,"Estaba Rodrigo del Leeds pero es verdad que no ha llegado a ser lo que se esperaba. \nLo que le falta es un delantero alto con movilidad para salir de la posicion y crear espacios a su espalda. Pero no hay nada ningun proyecto de ese tipo, ahora mismo (Rafa Mira?)."


In [40]:
if normalize:
    norm = data.merge(norm, on='tweet_id', how='left')
    norm.head(1)

,query_topic,verbs,text_orig,lemmad,word_pos,retweet_reply_like_quote,created_at,tweet_location,user_location,username,user_tweet_count,followers_following_listed,tweet_id,text_normd
0,esperar,['esperar'],"@Teleprensa33 @camarasal La miseria de salarios no espera análisis técnicos, son unos buitres estos empresarios explotadores.",el miseria de salario no esperar analisi tecnico ser uno buitre este empresario explotador,"{'la': 'DET', 'miseria': 'NOUN', 'de': 'ADP', 'salarios': 'NOUN', 'no': 'ADV', 'espera': 'VERB', 'analisis': 'NOUN', 'tecnicos': 'ADJ', 'son': 'AUX', 'unos': 'DET', 'buitres': 'NOUN', 'estos': 'DET', 'empresarios': 'NOUN', 'explotadores': 'ADJ'}","(0, 0, 0, 0)",2021-07-06 22:12:54,"Ayutuxtepeque, El Salvador","San Salvador, El Salvador",quintanilla503s,4196,"(108, 685, 0)",1412535051275940096,"La miseria de salarios no espera analisis tecnicos, son unos buitres estos empresarios explotadores."
1,esperar,['esperar'],"@lavigi @Ramarmo Estaba Rodrigo del Leeds pero es verdad que no ha llegado a ser lo que se esperaba. \nLo que le falta es un delantero alto con movilidad para salir de la posición y crear espacios a su espalda. Pero no hay nada ningún proyecto de ese tipo, ahora mismo (Rafa Mira?).",estar Rodrigo del Leeds pero ser verdad que no haber llegar a ser él que él esperar \n él que él faltar ser uno delantero alto con movilidad para salir de el posicion y crear espacio a su espalda pero no haber nada ningun proyecto de ese tipo ahora mismo Rafa Mira,"{'estaba': 'VERB', 'rodrigo': 'PROPN', 'del': 'ADP', 'leeds': 'PROPN', 'pero': 'CCONJ', 'es': 'AUX', 'verdad': 'NOUN', 'que': 'PRON', 'no': 'ADV', 'ha': 'AUX', 'llegado': 'VERB', 'a': 'ADP', 'ser': 'AUX', 'lo': 'PRON', 'se': 'PRON', 'esperaba': 'VERB', '\n': 'SPACE', 'le': 'PRON', 'falta': 'VERB', 'un': 'DET', 'delantero': 'NOUN', 'alto': 'ADJ', 'con': 'ADP', 'movilidad': 'NOUN', 'para': 'ADP', 'salir': 'VERB', 'de': 'ADP', 'la': 'DET', 'posicion': 'NOUN', 'y': 'CCONJ', 'crear': 'VERB', 'espacios': 'NOUN', 'su': 'DET', 'espalda': 'NOUN', 'hay': 'AUX', 'nada': 'PRON', 'ningun': 'DET', 'proyecto': 'NOUN', 'ese': 'DET', 'tipo': 'NOUN', 'ahora': 'ADV', 'mismo': 'ADV', 'rafa': 'PROPN', 'mira': 'PROPN'}","(0, 1, 2, 0)",2021-07-06 22:11:57,"Paris, France","Paris, France",AlmendraPeleona,14057,"(1097, 354, 8)",1412534812712260096,"Estaba Rodrigo del Leeds pero es verdad que no ha llegado a ser lo que se esperaba. \nLo que le falta es un delantero alto con movilidad para salir de la posicion y crear espacios a su espalda. Pero no hay nada ningun proyecto de ese tipo, ahora mismo (Rafa Mira?)."
2,esperar,['esperar'],@SuryZury01 Bueno espero serlo,bueno esperar ser él,"{'bueno': 'INTJ', 'espero': 'VERB', 'serlo': 'AUX'}","(0, 0, 2, 0)",2021-07-06 22:10:37,Honduras,"Tegucigalpa, Honduras",mrhombre_,15113,"(4657, 1217, 2)",1412534477876781056,Bueno espero serlo


In [43]:
if normalize:
    data = norm[['query_topic', 'verbs', 'text_orig', 'text_normd', 'lemmad', 'word_pos', 'retweet_reply_like_quote', 'created_at', 'tweet_location', 'user_location', 'username', 'user_tweet_count', 'followers_following_listed', 'tweet_id']].copy()
    data.head(1)

,query_topic,verbs,text_orig,text_normd,lemmad,word_pos,retweet_reply_like_quote,created_at,tweet_location,user_location,username,user_tweet_count,followers_following_listed,tweet_id
0,esperar,['esperar'],"@Teleprensa33 @camarasal La miseria de salarios no espera análisis técnicos, son unos buitres estos empresarios explotadores.","La miseria de salarios no espera analisis tecnicos, son unos buitres estos empresarios explotadores.",el miseria de salario no esperar analisi tecnico ser uno buitre este empresario explotador,"{'la': 'DET', 'miseria': 'NOUN', 'de': 'ADP', 'salarios': 'NOUN', 'no': 'ADV', 'espera': 'VERB', 'analisis': 'NOUN', 'tecnicos': 'ADJ', 'son': 'AUX', 'unos': 'DET', 'buitres': 'NOUN', 'estos': 'DET', 'empresarios': 'NOUN', 'explotadores': 'ADJ'}","(0, 0, 0, 0)",2021-07-06 22:12:54,"Ayutuxtepeque, El Salvador","San Salvador, El Salvador",quintanilla503s,4196,"(108, 685, 0)",1412535051275940096


In [45]:
if normalize:
    data.to_excel(path_save/file_name_folder/file_name_data, index=False)

### Split Verbs

In [121]:
# Convert verbs column from string to list
data['verbs'] = data['verbs'].apply(eval)

In [128]:
multi = data['verbs'].apply(len)>1
print(f'{multi.sum()} tokens have >1 desired verbs')

12577 tokens have >1 desired verbs


In [143]:
data = data.explode('verbs')
print(f'{data.shape[0]} exploded tokens')

60843 exploded tokens


In [144]:
n['verbs'].value_counts()

esperar       43826
ver            3289
decir          2677
querer         1913
saber          1252
creer          1144
sentir          700
pensar          637
entender        532
pedir           488
parecer         473
mandar          396
desear          360
contar          342
lograr          313
recordar        261
responder       217
conseguir       163
alegrar         162
imaginar        153
demostrar       143
mostrar         121
suponer         113
preocupar       108
acordar          94
dudar            80
lamentar         77
considerar       73
recomendar       62
negar            61
gritar           60
confirmar        57
prometer         46
solicitar        46
jurar            42
reclamar         37
mencionar        37
insistir         37
asegurar         36
apostar          34
ordenar          27
temer            26
admitir          26
confesar         19
rogar            18
sugerir          14
comprobar        13
afirmar          12
adivinar         11
prever           10


### Finding Occurrences of 'que'

In [151]:
"""
Convert text to a Markdown object with the desired string (or 
  regex pattern) @hl highlighted  
"""
def hilite(text, hl):
    for m in set(re.findall(hl, text)):
        pat = rf'\b{m}\b'
        text = re.sub(pat, f'<span style="color: #ff0000">{m}</span>', text)
        
    return Markdown(text)

In [176]:
def has_que(token:pd.Series) -> bool:
    pat = rf'\b({token["verbs"]} (que|q))\b'
    
    return len(re.findall(pat, token['lemmad']))>0

In [177]:
que_pat = r'\b(que|q)\b'

In [ ]:
has_que = data[data['text_normd'].str.contains(que_pat, regex=True)]

In [178]:
has_que = data.apply(has_que, axis=1).rename('has_que')
print(f'{round(has_que.sum()/data.shape[0], 2)} contain desired \'que\' -- {has_que.sum()} out of {data.shape[0]}')

0.32 contain desired 'que' -- 19372 out of 60843


In [179]:
work = pd.concat([data[['tweet_id', 'verbs', 'text_normd', 'lemmad']], has_que], axis=1)
work.head(1)

,tweet_id,verbs,text_normd,lemmad,has_que
0,1412535051275940096,esperar,"La miseria de salarios no espera analisis tecnicos, son unos buitres estos empresarios explotadores.",el miseria de salario no esperar analisi tecnico ser uno buitre este empresario explotador,False


In [187]:
n = work.apply(lambda token: hilite(token['lemmad'], token['verbs']), axis=1)
for a in n[work['has_que']==True].iloc[-5:]:
    display(a)

<span style="color: #ff0000">esperar</span> que él pronto JAJAJA

junio ser y ser el peor mes del ano hasta aqui para yo despertar por el manana y acostar él por el noche en el mismo situacion sin gana sin motivacion con angustia y a veces con bronca solo <span style="color: #ff0000">esperar</span> que terminar como si ese cambiar algo


 tener prueba que <span style="color: #ff0000">demostrar</span> que no ser asi nunca reprogramar uno turno repetir ser uno delito el uso de el imagen publico sin consentimiento y no ir a soportar que él usar con fin politico 
 esperar disculpa Publicas estar asustado no ser asi

no tener explicacion como el mil de asegurado no reaccionar aun y salir a rodear todo el instalación del IPS para frenar su vaciamiento malo atencion estar <span style="color: #ff0000">ver</span> que haber uno plan de fundir el previsional mientras solo esperar que ocurrir

no tener explicacion como el mil de asegurado no reaccionar aun y salir a rodear todo el instalación del IPS para frenar su vaciamiento malo atencion estar ver que haber uno plan de fundir el previsional mientras solo <span style="color: #ff0000">esperar</span> que ocurrir

### Location Distribution

In [24]:
import os

path_places = (f for f in os.listdir(data_prefix+data_dir) if 'places' in f)

In [25]:
place_data = pd.DataFrame()

for path in path_places:
    try:
        place_data = place_data.append(
            pd.read_csv(data_prefix+data_dir+path, sep='~'), 
            ignore_index=True)
    except Exception as e:
        print(e.args)

In [26]:
place_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37812 entries, 0 to 37811
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   country   37812 non-null  object
 1   location  37812 non-null  object
 2   place_id  37812 non-null  object
dtypes: object(3)
memory usage: 886.3+ KB


In [27]:
place_dups = place_data.duplicated(subset='place_id')
place_dups.sum()

33203

In [32]:
place_data.drop(place_data[place_dups].index, inplace=True)
print(place_data.shape[0])
display(place_data.head(3))

4609


/tmp/ipykernel_86075/4200568867.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  place_data.drop(place_data[place_dups].index, inplace=True)


,country,location,place_id
0,España,"Alhaurín el Grande, España",02798a66b5842b79
1,Paraguay,"Central, Paraguay",016ea1445db12efb
2,Colombia,"Bogotá, D.C., Colombia",0161be1b3f98d6c3


In [31]:
place_data.to_csv(data_prefix+data_dir+'places-cleaned.csv', index=False)

In [36]:
data = pd.read_csv(data_prefix+data_dir+data_path_good, sep='~')
data.head(3)

,tweet_id,verbs,word_pos,lemmad,lang,text_orig,created_at,referenced_tweet_id,author_id,mentioned_user_id,place_id,query_topic,retweet_reply_like_quote,username,user_location,user_tweet_count,followers_following_listed,tweet_location
0,1412535051275939848,['esperar'],"{'la': 'DET', 'miseria': 'NOUN', 'de': 'ADP', ...",el miseria de salario no esperar analisi tecni...,es,@Teleprensa33 @camarasal La miseria de salario...,2021-07-06 22:12:54,['1412532799492399108'],908149252928495621,"['800664319', '201882822']",00a3108d44a35f06,esperar,"(0, 0, 0, 0)",quintanilla503s,"San Salvador, El Salvador",4196,"(108, 685, 0)","Ayutuxtepeque, El Salvador"
1,1412534812712259591,['esperar'],"{'estaba': 'VERB', 'rodrigo': 'PROPN', 'del': ...",estar Rodrigo del Leeds pero ser verdad que no...,es,@lavigi @Ramarmo Estaba Rodrigo del Leeds pero...,2021-07-06 22:11:57,['1412533286354751490'],923999535977959429,"['13882822', '599612830']",09f6a7707f18e0b1,esperar,"(0, 1, 2, 0)",AlmendraPeleona,"Paris, France",14057,"(1097, 354, 8)","Paris, France"
2,1412534477876781067,['esperar'],"{'bueno': 'INTJ', 'espero': 'VERB', 'serlo': '...",bueno esperar ser él,es,@SuryZury01 Bueno espero serlo,2021-07-06 22:10:37,['1412517081078173699'],270355529,['933848051088789505'],d862adecd71aa2a2,esperar,"(0, 0, 2, 0)",mrhombre_,"Tegucigalpa, Honduras",15113,"(4657, 1217, 2)",Honduras


In [38]:
data = data.merge(place_data[['place_id', 'country']], how='left', on='place_id')
data.head()

,tweet_id,verbs,word_pos,lemmad,lang,text_orig,created_at,referenced_tweet_id,author_id,mentioned_user_id,place_id,query_topic,retweet_reply_like_quote,username,user_location,user_tweet_count,followers_following_listed,tweet_location,country
0,1412535051275939848,['esperar'],"{'la': 'DET', 'miseria': 'NOUN', 'de': 'ADP', ...",el miseria de salario no esperar analisi tecni...,es,@Teleprensa33 @camarasal La miseria de salario...,2021-07-06 22:12:54,['1412532799492399108'],908149252928495621,"['800664319', '201882822']",00a3108d44a35f06,esperar,"(0, 0, 0, 0)",quintanilla503s,"San Salvador, El Salvador",4196,"(108, 685, 0)","Ayutuxtepeque, El Salvador",El Salvador
1,1412534812712259591,['esperar'],"{'estaba': 'VERB', 'rodrigo': 'PROPN', 'del': ...",estar Rodrigo del Leeds pero ser verdad que no...,es,@lavigi @Ramarmo Estaba Rodrigo del Leeds pero...,2021-07-06 22:11:57,['1412533286354751490'],923999535977959429,"['13882822', '599612830']",09f6a7707f18e0b1,esperar,"(0, 1, 2, 0)",AlmendraPeleona,"Paris, France",14057,"(1097, 354, 8)","Paris, France",France
2,1412534477876781067,['esperar'],"{'bueno': 'INTJ', 'espero': 'VERB', 'serlo': '...",bueno esperar ser él,es,@SuryZury01 Bueno espero serlo,2021-07-06 22:10:37,['1412517081078173699'],270355529,['933848051088789505'],d862adecd71aa2a2,esperar,"(0, 0, 2, 0)",mrhombre_,"Tegucigalpa, Honduras",15113,"(4657, 1217, 2)",Honduras,Honduras
3,1412534445521915905,['esperar'],"{'yo': 'PRON', 'como': 'SCONJ', 'la': 'DET', '...",yo como el mayoria de yo no esperar que el sel...,es,"Yo, como la mayoría de nosotros, no esperaba q...",2021-07-06 22:10:30,NaN,887438330,NaN,d151b58850d4cd1e,esperar,"(0, 0, 0, 0)",GermanMoleroo,Barrio Peral City,31326,"(856, 1069, 15)","Cartagena, España",España
4,1412534411233566731,['esperar'],"{'mi': 'DET', 'mas': 'ADV', 'sincero': 'ADJ', ...",mi mas sincero apoyo a el libertad de expresio...,es,@salvadorilla @eljueves Mi más sincero apoyo a...,2021-07-06 22:10:21,['1412436417716891652'],714833865,"['233240412', '17191272']",fdfe71d3b7d78b56,esperar,"(0, 0, 0, 0)",hernanmilla72,NaN,4033,"(233, 862, 0)","Arganda del Rey, España",España


In [52]:
data['country'].unique()

array(['El Salvador', 'France', 'Honduras', 'España', 'Argentina',
       'United States', 'Portugal', 'México', 'Ecuador', 'Colombia',
       'Chile', 'Venezuela', 'Uruguay', 'Panama', 'Polska', 'Nicaragua',
       'Dominican Republic', 'Peru', 'Brasil', 'Bolivia', 'Guatemala',
       'Costa Rica', 'Italia', 'Deutschland', 'United Kingdom',
       'Paraguay', 'New Zealand', 'Andorra', 'Canada', 'Australia',
       'Cuba', 'Ελλάς', '日本', 'ישראל', 'Nederland', 'België', 'Danmark',
       'Sverige', 'Kosovë', 'Schweiz', 'المملكة العربية السعودية',
       'Royaume du Maroc', 'Ireland', 'România', 'Türkiye', 'Kenya',
       'Malta', 'Equatorial Guinea', 'Republika ng Pilipinas',
       'Bonaire, Sint Eustatius and Saba', 'Norge', 'Suomi',
       'Magyarország', 'Österreich', 'Россия', 'Việt Nam', 'Curaçao',
       '대한민국', 'Malaysia', 'България', 'Saint-Martin', 'Ethiopia',
       'الامارات العربية المتحدة', 'Lietuva', 'Aruba', '新加坡', 'Srbija',
       'भारत', 'Uganda', 'République Démocrati

In [51]:
data.value_counts(subset='country')

country
España       9358
Argentina    8412
México       7023
Colombia     3821
Chile        2990
             ... 
Indonesia       1
Jamaica         1
Kosovë          1
Latvija         1
Algérie         1
Length: 94, dtype: int64

In [47]:
# Turn verbs to list
def str_to_list(string):
    l = re.sub(r'[\[\]\']+', '', string).split(', ')
    return l

In [62]:
merged.drop(columns='Unnamed: 0', inplace=True)

In [63]:
merged['verbs'] = merged['verbs'].apply(str_to_list)

TypeError: expected string or bytes-like object

In [64]:
merged.head(3)

,query_topic,verbs,text_orig,word_pos,retweet_reply_like_quote,created_at,tweet_location,user_location,username,user_tweet_count,followers_following_listed,tweet_id
0,peru,[esperar],@meiermq Hay peruanos que sufren de miopía. An...,"{'hay': 'AUX', 'peruanos': 'NOUN', 'que': 'SCO...","(0, 0, 0, 0)",2021-06-12 01:18:03,"Santiago de Surco, Peru",NaN,luchooviedo3,971,"(5, 99, 0)",1403521950018937088
1,peru,[decir],"@joaquin1203 Mano, yo sentí lo mismo pero con ...","{'mano': 'INTJ', 'yo': 'PRON', 'senti': 'VERB'...","(0, 0, 0, 0)",2021-06-12 01:17:08,"Chimbote, Peru","Lima, Peru",MPQSR,111447,"(26268, 1097, 76)",1403521717369295104
2,peru,[ver],"@vickyperiodista Lloró viendo mi Perú,como una...","{'lloro': 'VERB', 'viendo': 'VERB', 'mi': 'DET...","(0, 0, 0, 0)",2021-06-12 01:16:27,"La Molina, Peru",NaN,HuamanguinaR,724,"(3, 30, 0)",1403521546090651904


In [65]:
def get_verb_type(verbs, kind):
    v = []
    
    if kind=='s':
        v = es_verbs_stative
    elif kind=='v':
        v = es_verbs_volit
    elif kind=='e':
        v = es_verbs_epistemic
    else:
        raise KeyError('Invalid verb type! '\
                       'Must be one of \'s\': stative, \'v\': volitional, \'e\': epistemic')
    
    for verb in verbs:
        if verb in v:
            return True
        
    return False

In [66]:
s = merged['verbs'].apply(get_verb_type, kind='s')
v = merged['verbs'].apply(get_verb_type, kind='v')
e = merged['verbs'].apply(get_verb_type, kind='e')

merged_stat = merged.loc[merged[s].index]
merged_volit = merged.loc[merged[v].index]
merged_epi = merged.loc[merged[e].index]

In [68]:
display(merged_stat.head(3))
display(merged_volit.head(3))
display(merged_epi.head(3))

,query_topic,verbs,text_orig,word_pos,retweet_reply_like_quote,created_at,tweet_location,user_location,username,user_tweet_count,followers_following_listed,tweet_id
1,peru,[decir],"@joaquin1203 Mano, yo sentí lo mismo pero con ...","{'mano': 'INTJ', 'yo': 'PRON', 'senti': 'VERB'...","(0, 0, 0, 0)",2021-06-12 01:17:08,"Chimbote, Peru","Lima, Peru",MPQSR,111447,"(26268, 1097, 76)",1403521717369295104
2,peru,[ver],"@vickyperiodista Lloró viendo mi Perú,como una...","{'lloro': 'VERB', 'viendo': 'VERB', 'mi': 'DET...","(0, 0, 0, 0)",2021-06-12 01:16:27,"La Molina, Peru",NaN,HuamanguinaR,724,"(3, 30, 0)",1403521546090651904
4,peru,"[recordar, decir]",@MilagrosLeivaG @FSagasti @MartinVizcarraC @JN...,"{'es': 'AUX', 'lo': 'PRON', 'unico': 'ADJ', ' ...","(0, 0, 0, 0)",2021-06-12 01:02:40,"Surquillo, Peru",NaN,r2mirta,13337,"(26, 27, 0)",1403518075937464064


,query_topic,verbs,text_orig,word_pos,retweet_reply_like_quote,created_at,tweet_location,user_location,username,user_tweet_count,followers_following_listed,tweet_id
0,peru,[esperar],@meiermq Hay peruanos que sufren de miopía. An...,"{'hay': 'AUX', 'peruanos': 'NOUN', 'que': 'SCO...","(0, 0, 0, 0)",2021-06-12 01:18:03,"Santiago de Surco, Peru",NaN,luchooviedo3,971,"(5, 99, 0)",1403521950018937088
3,peru,[querer],@crebosio95 Vargas Llosa es un narcisista\nQue...,"{'vargas': 'PROPN', 'llosa': 'PROPN', 'es': 'A...","(10, 1, 82, 0)",2021-06-12 01:11:25,"Santiago de Surco, Peru",NaN,DraGonzalez,27910,"(49108, 210, 32)",1403520281218519040
8,peru,[esperar],@pintoEcuador Cuando el Mameluco se apropio de...,"{'cuando': 'SCONJ', 'el': 'DET', 'mameluco': '...","(1, 1, 1, 0)",2021-06-12 00:52:30,"Quito, Ecuador",NaN,renesanchezcor1,6155,"(29, 140, 0)",1403515518993276928


,query_topic,verbs,text_orig,word_pos,retweet_reply_like_quote,created_at,tweet_location,user_location,username,user_tweet_count,followers_following_listed,tweet_id
4,peru,"[recordar, decir]",@MilagrosLeivaG @FSagasti @MartinVizcarraC @JN...,"{'es': 'AUX', 'lo': 'PRON', 'unico': 'ADJ', ' ...","(0, 0, 0, 0)",2021-06-12 01:02:40,"Surquillo, Peru",NaN,r2mirta,13337,"(26, 27, 0)",1403518075937464064
9,peru,"[sentir, reclamar]",@Avelino_Guillen @VLADIMIR_CERRON @JNE_Peru Dr...,"{'dr.': 'PROPN', 'no': 'ADV', 'se': 'PRON', 't...","(0, 0, 1, 0)",2021-06-12 00:50:30,"Magdalena Vieja, Peru","Jesús Maria, Peru",juliet_navarro,3669,"(64, 224, 1)",1403515013919396096
10,peru,[saber],@japt1987 @parocolectiva @JNE_Peru Mafia tiene...,"{'mafia': 'PROPN', 'tiene': 'VERB', 'la': 'DET...","(0, 1, 1, 0)",2021-06-12 00:49:02,"Callao, Peru",NaN,lui77841331,2527,"(7, 64, 0)",1403514645516848896


In [67]:
print(f'Stative: {merged_stat.shape[0]}'\
      f'\nEpistemic: {merged_epi.shape[0]}'\
      f'\nVolitional: {merged_volit.shape[0]}')

Stative: 9084
Epistemic: 7407
Volitional: 5772


In [69]:
# Save as .xlsx
merged_stat.to_excel(save_prefix+'pos-tagged-sample-stative.xlsx')
merged_epi.to_excel(save_prefix+'pos-tagged-sample-epistemic.xlsx')
merged_volit.to_excel(save_prefix+'pos-tagged-sample-volitional.xlsx')

### Dependency Trees

In [93]:
from spacy import displacy

In [72]:
processed.head(3)

,verbs,tweet_id,text_normd
0,[esperar],1403521950018936837,"(Hay, peruanos, que, sufren, de, miopia, ., An..."
1,[decir],1403521717369294852,"(Mano, ,, yo, senti, lo, mismo, pero, con, el,..."
2,[ver],1403521546090651654,"(Lloro, viendo, mi, Peru, ,, como, una, mujer,..."


In [79]:
sample = processed['text_normd'][:5]
sample

0    (Hay, peruanos, que, sufren, de, miopia, ., An...
1    (Mano, ,, yo, senti, lo, mismo, pero, con, el,...
2    (Lloro, viendo, mi, Peru, ,, como, una, mujer,...
3    (Vargas, Llosa, es, un, narcisista, \n, Queria...
4    (Es, lo, unico,  , que, ha, echo, bien, ,, por...
Name: text_normd, dtype: object

In [102]:
displacy.render(sample, style='dep')

### Identifying Correlations